In [1]:
import math
import random
import numpy as np
import matplotlib.pyplot as plt
%matplotlib inline

In [2]:
class Value:
    
    def __init__(self, data, _children=(), _op=(), label=None):
        self.data = data
        self.grad = 0
        self._backward = lambda: None
        self._prev = set(_children)
        self._op = _op
        self.label = label
    
    def __repr__(self):
        return f"Value(data={self.data}, label={self.label})"
    
    def __add__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data + other.data, (self, other), '+')
        
        def _backward():
            self.grad += 1.0 * out.grad
            other.grad += 1.0 * out.grad
            
        out._backward = _backward
        return out
    
    def __mul__(self, other):
        other = other if isinstance(other, Value) else Value(other)
        out = Value(self.data * other.data, (self, other), '*')
        
        def _backward():
            self.grad += other.data * out.grad
            other.grad += self.data * out.grad
        
        out._backward = _backward
        return out
    
    def __pow__(self, other):
        assert isinstance(other, (int, float)), "only supports int/float for now"
        out = Value(self.data**other, (self,), f'**{other}')
        
        def _backward():
            self.grad += other * (self.data ** (other - 1)) * out.grad
        
        out._backward = _backward
        return out
    
    def __radd__(self, other): # other + self
        return self + other

    def __rmul__(self, other): # other * self
        return self * other
    
    def __truediv__(self, other): # self / other
        return self * other**-1
    
    def __neg__(self): # -self
        return self * -1
    
    def __sub__(self, other): # self - other
        return self + (-other)
    
    def tanh(self):
        x = self.data
        t = (math.exp(2*x) - 1)/(math.exp(2*x) + 1)
        out = Value(t, (self,), 'tanh')
        
        def _backward():
            self.grad += (1 - t**2) * out.grad
        
        out._backward = _backward
        return out
    
    def exp(self):
        x = self.data
        out = Value(math.exp(x), (self,), 'exp')
        
        def _backward():
            self.grad += out.data * out.grad
        
        out._backward = _backward
        return out 
    
    def backward(self):
        def topo_sort(node):
            topo = []
            visited = set()
            def build_topo(v):
                if v not in visited:
                    visited.add(v)
                    for child in v._prev:
                        build_topo(child)
                    topo.append(v)
            build_topo(node)
            return topo
        
        self.grad = 1.0
        for node in reversed(topo_sort(self)):
            node._backward()

In [3]:
class Neuron:
    
    def __init__(self, nin):
        self.w = [Value(random.uniform(-1,1)) for _ in range(nin)]
        self.b = Value(random.uniform(-1,1))
        
    def __call__(self, x):
        # w * x + b
        act = sum((wi*xi for wi,xi in zip(self.w, x, strict=True)), self.b)
        out = act.tanh()
        return out
    
    def parameters(self):
        return self.w + [self.b] # list

class Layer:
    
    def __init__(self, nin, nout):
        self.neurons = [Neuron(nin) for _ in range(nout)]
        
    def __call__(self, x):
        outs = [n(x) for n in self.neurons]
        return outs[0] if len(outs) == 1 else outs

    def parameters(self):
        return [p for neuron in self.neurons for p in neuron.parameters()]

class MLP:
    
    def __init__(self, nin, nouts):
        sz = [nin] + nouts
        self.layers = [Layer(sz[i], sz[i+1]) for i in range(len(nouts))]
        
    def __call__(self, x):
        for layer in self.layers:
            x = layer(x)
        return x

    def parameters(self):
        return [p for layer in self.layers for p in layer.parameters()]

In [4]:
x = [2.0, 3.0, 4.0]
n = MLP(3, [4,4,1])
n(x)

Value(data=0.8247430393912225, label=None)

In [5]:
# 3 dim dataset
xs = [
    [2.0, 3.0, -1.0],
    [3.0, -1.0, 0.5],
    [0.5, 1.0, 1.0],
    [1.0, 1.0, -1.0],
]
ys = [1.0, -1.0, -1.0, 1.0] # targets

In [6]:
# predictions is just the forward
ypreds = [n(x) for x in xs]
ypreds

[Value(data=0.5565857642201866, label=None),
 Value(data=0.8370883746666125, label=None),
 Value(data=0.8518424546067377, label=None),
 Value(data=0.4234389498308731, label=None)]

In [7]:
def mse(targets, preds):
    return sum((p - t)**2 for t,p in zip(targets, preds))

In [8]:
loss = mse(ys, ypreds)
loss

Value(data=7.333253002083445, label=None)

In [9]:
loss.backward()

In [10]:
n.layers[0].neurons[0].w[0]

Value(data=0.17134970733125865, label=None)

In [11]:
n.layers[0].neurons[0].w[0].grad

-0.1781867196420085

In [12]:
# time to nudge the parameters

In [13]:
step_size = 0.01

# gradient points in the direction of increasing the loss, hence the negative step size

for p in n.parameters():
    p.data += -step_size * p.grad

In [14]:
ypreds = [n(x) for x in xs]
ypreds

[Value(data=0.5749601635877541, label=None),
 Value(data=0.8227332350107972, label=None),
 Value(data=0.8441689032590021, label=None),
 Value(data=0.4429211034751062, label=None)]

In [15]:
loss = mse(ys, ypreds)
loss

Value(data=7.21431114925118, label=None)

In [16]:
# we can see that the loss is decreasing
# gradient descent is basically this in an iteration

# forward pass -> backward pass (calc gradients) -> update (use gradients to nudge the parameters) ---> ...

In [17]:
# continue running the gradient descent method until you have a very low loss

In [18]:
# update
step_size = 0.01
for p in n.parameters():
    p.data += -step_size * p.grad
# forward
ypreds = [n(x) for x in xs]
# backward
loss = mse(ys, ypreds)
loss.backward()

loss

Value(data=7.095996656809492, label=None)

In [19]:
ypreds

[Value(data=0.5916215060323687, label=None),
 Value(data=0.8074055488614219, label=None),
 Value(data=0.8363863170293772, label=None),
 Value(data=0.4613032958645057, label=None)]